In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#import shapefile as shp
from collections import defaultdict

### Set Parameters

In [2]:
M = 100000
maxOpenD = 50
maxOpenA = 20
maxDistance = 10000

maxDist_D = 150
maxDist_A = 350

#maxPopD- max capacity of each district court
#maxPopA - max capacity of each appeals court

#xS,yS 
#xD,yD
#xA,yA

#langS
#langD
#langA


#param d{i in Settlements,j in DistrictCourts} 
#	:= sqrt( (xS[i]-xD[j])^2 + (yS[i]-yD[j])^2); #distance between settlement and district court
#param a{i in Settlements,k in AppealsCourts} 
#	:= sqrt( (xA[k]-xS[i])^2 + (yA[k]-yS[i])^2); #distance between district court and appeals court	

### Extract Data

In [3]:
#Get Settlement List
Settlements = pd.read_csv("afg_ppl_settlement_pnt.csv",sep=",")
Settlements = Settlements[[0,10,11]]
S = Settlements.shape[0]
Settlement_List = Settlements['OBJECTID'].tolist()

#Get District Court List
Districts = pd.read_csv("District_Courts.csv",sep=",")
Districts = Districts[[1,4,5]]
D = Districts.shape[0]
District_List = Districts['DIST_CODE'].tolist()

#Get Appeals Court List
Appeals = pd.read_csv("Appeals_Courts.csv",sep=",")
Appeals = Appeals[[1,4,5]]
A = Appeals.shape[0]
Appeals_List = Appeals['PROV_CODE'].tolist()

#### Create Data Subset for Settlements

#### ------------------------------------------------

In [4]:
Settlements = Settlements.sample(frac = 0.001, replace = False)
S = Settlements.shape[0]
Settlement_List = Settlements['OBJECTID'].tolist()

####  ------------------------------------------------

In [5]:
#Create Dictionaries
Settlement_Dict = Settlements.set_index('OBJECTID').T.to_dict('list')
District_Dict = Districts.set_index('DIST_CODE').T.to_dict('list')
Appeals_Dict = Appeals.set_index('PROV_CODE').T.to_dict('list')

In [6]:
#Create Dictionaries for District Courthouse Distances

Dist_D = {}

R = 6371e3


for d in District_List:
    d_lon = District_Dict[d][0]
    d_lat = District_Dict[d][1]
    theta2 = np.radians(d_lat)


    for s in Settlement_List:
        s_lon = Settlement_Dict[s][1]
        s_lat = Settlement_Dict[s][0]
        theta1 = np.radians(s_lat)
        
        #Distance to District Court
        delta_theta = np.radians(d_lat - s_lat)
        delta_lambda = np.radians(d_lon - s_lon)
        a = np.sin(delta_theta/2) * np.sin(delta_theta/2) + np.cos(theta1) * np.cos(theta2) * np.sin(delta_lambda/2) * np.sin(delta_lambda/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        
        Dist_D[s,d] = (R * c)/1000
    

In [7]:
#Create Dictionaries for Appeals Courthouse Distances

Dist_A = {}

R = 6371e3

for a in Appeals_List:
    a_lon = Appeals_Dict[a][0]
    a_lat = Appeals_Dict[a][1]
    theta3 = np.radians(a_lat)

    for s in Settlement_List:
        s_lon = Settlement_Dict[s][1]
        s_lat = Settlement_Dict[s][0]
        theta1 = np.radians(s_lat)
        
        #Distance to District Court
        delta_theta = np.radians(a_lat - s_lat)
        delta_lambda = np.radians(a_lon - s_lon)
        a1 = np.sin(delta_theta/2) * np.sin(delta_theta/2) + np.cos(theta1) * np.cos(theta3) * np.sin(delta_lambda/2) * np.sin(delta_lambda/2)
        c = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1-a1))
        
        Dist_A[s,a] = (R * c)/1000

##### Helper function

In [8]:
# Return value of variable
def VarVal(var):
    if (type(var) == gurobipy.Var): #check if gurobi variable
        val = var.X
    else:
        val = 0
    return val

# ===========================================

### Create Model - LP

In [9]:
#Create Model
LP = Model("Afg_LP")

LP.Params.OutputFlag = 1  #Suppress output
#LP.Params.LazyConstraints = 1

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1


### Create Variables - LP

##### e_ijk - LP

In [10]:
# Create e_i_j_k variables
e = {}
for i in Settlement_List:
    e[i] = {}
    for j in District_List:
        e[i][j] = {}
        for k in Appeals_List:
            if (Dist_D[i,j] < maxDist_D) & (Dist_A[i,k] < maxDist_A):
                e[i][j][k] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='e_%s_%s_%s' % (i, j, k))
            else:
                e[i][j][k] = 0
LP.update()

##### e_ikj index rearrangement

In [11]:
# Initialize Dict
e_j = {}
for i in Settlement_List:
    e_j[i] = {}
    for k in Appeals_List:
        e_j[i][k] = {}

In [12]:
for s_key, d_dict in e.items():
    for d_key, a_dict in d_dict.items():
        for a_key, val in a_dict.items():
            e_j[s_key][a_key][d_key] = val

##### d_ij - LP

In [13]:
# Initialize Dict
d = {}
for i in Settlement_List:
    d[i] = {}
    for j in District_List:
        d[i][j] = {}

In [14]:
#Aggregate e_ijk across k
for i in Settlement_List:
    for j in District_List:
        d[i][j] = quicksum(e[i][j][k] for k in Appeals_List)

##### a_ik - LP

In [15]:
# Initialize Dict
a = {}
for i in Settlement_List:
    a[i] = {}
    for k in Appeals_List:
        a[i][k] = {}

In [16]:
#Aggregate e_ijk across d
for i in Settlement_List:
    for k in Appeals_List:
        a[i][k] = quicksum(e[i][j][k] for j in District_List)

##### c_jk - LP

In [17]:
# Create c_j_k variables
c = {}
for j in District_List:
    c[j] = {}
    for k in Appeals_List:
        c[j][k] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=0, name='c_%s_%s' % (j, k))
LP.update()

In [18]:
# Create c_j_k transpose dictionary 
c_t = defaultdict(dict)
for key, val in c.items():
    for subkey, subval in val.items():
        c_t[subkey][key] = subval

##### openD - LP

In [19]:
#Create openD LP variables
openD_LP = {}
for j in District_List:
    openD_LP[j] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='openD_LP_%s' % (j))
LP.update()

##### openA - LP

In [20]:
#Create openA LP variables
openA_LP = {}
for k in Appeals_List:
    openA_LP[k] = LP.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='openA_LP_%s' % (k))
LP.update()

### Create Constraints - LP

#### D_ij row sums and column sums - LP

In [21]:
#One S -> D Assignment LP
for i in Settlement_List:
    LP.addConstr(quicksum(d[i][j] for j in d[i]) == 1)
LP.update()

In [22]:
#Maximum District Courts Open LP
for i in Settlement_List:
    for j in District_List:
        if (type(d[i][j]) != int):
            LP.addConstr(d[i][j] <= openD_LP[j])
LP.update()

#### A_ik row sums and column sums - LP

In [23]:
#One S -> A Assignment LP
for i in Settlement_List:
    LP.addConstr(quicksum(a[i][k] for k in Appeals_List) == 1)
LP.update()

In [24]:
#Maximum Appeals Courts Open LP
for i in Settlement_List:
    for k in Appeals_List:
        if (type(a[i][k]) != int):
            LP.addConstr(a[i][k] <= openA_LP[k])
LP.update()

#### C_jk row sums and column sums - LP

In [25]:
#One D -> A Assignment LP
for j in District_List:
    LP.addConstr(quicksum(c[j][k] for k in c[j]) == openD_LP[j])
LP.update()

In [26]:
#C constraint LP
for j in District_List:
    for k in Appeals_List:
        LP.addConstr(c[j][k] <= openA_LP[k])
LP.update()

#### Linking Constraints

In [ ]:
for i in Settlement_List:
    for j in District_List:
        for k in Appeals_List:
            LP.addConstr(e[i][j][k] <= c[j][k])

##### Max Open Courthouse Constraints - LP

In [27]:
LP.addConstr(quicksum(openD_LP[j] for j in District_List) <= maxOpenD)
LP.addConstr(quicksum(openA_LP[k] for k in Appeals_List) <= maxOpenA)

LP.update()

### Set Objective Function - LP

#### D_ij and A_ik - LP

In [28]:
LP.setObjective(
        quicksum(   quicksum(   Dist_D[i,j]*d[i][j] for i in Settlement_List)    for j in District_List) + \
        quicksum(   quicksum(   Dist_A[i,k]*a[i][k] for i in Settlement_List)    for k in Appeals_List), GRB.MINIMIZE)
LP.update()

### Optimize

In [ ]:
violated_LP = 1

while violated_LP > 0:
    
    print("Optimize LP")
    LP.optimize()
    violated_LP = 0
    #Check for violated constraints
    for i in Settlement_List:
        for j in District_List:
            for k in Appeals_List:
                #Assign vars based on 0 or gurobi.Var
                print(e[i][j][k])
                print(c[j][k])
                e_val = VarVal(e[i][j][k])
                c_val = VarVal(c[j][k])
                
                #Add constraints if violated
                if (e_val > c_val):
                    violated_LP = violated_LP + 1
                    LP.addConstr(e[i][j][k] <= c[j][k])
    LP.update()


In [29]:
LP.optimize()

Optimize a model with 33986 rows, 66288 columns and 269814 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 33894 rows and 66288 columns
Presolve time: 0.05s

Solved with barrier
Solved in 0 iterations and 0.05 seconds
Infeasible model


In [ ]:
c[0][0].X

### Output Solution

In [ ]:
LP.write("out_LP.sol")
MIP.write("out_MIP.sol")

###  Plotting

#### DOUT

In [ ]:
# DOUT - District Assignments

DOUT = pd.DataFrame.from_dict({(i,j): VarVal(d[i][j]) 
                           for i in d.keys() 
                           for j in d[i].keys()},
                           orient='index')

Settlements_DOUT = [i[0] for i in DOUT.index]
Districts_DOUT = [i[1] for i in DOUT.index]

DOUT['Settlement'] = Settlements_DOUT
DOUT['DistrictCourts'] = Districts_DOUT

DOUT = DOUT[DOUT[0] == 1.0]

#### AOUT

In [ ]:
#AOUT - Appeals Assignments

AOUT = pd.DataFrame.from_dict({(i,j): VarVal(a[i][j]) 
                           for i in a.keys() 
                           for j in a[i].keys()},
                           orient='index')

Settlements_AOUT = [i[0] for i in AOUT.index]
Appeals_AOUT = [i[1] for i in AOUT.index]

AOUT['Settlement'] = Settlements_AOUT
AOUT['AppealsCourts'] = Appeals_AOUT

AOUT = AOUT[AOUT[0] == 1.0]

#### COUT

In [ ]:
#COUT - Appeals Assignments

COUT = pd.DataFrame.from_dict({(i,j): VarVal(c[i][j]) 
                           for i in c.keys() 
                           for j in c[i].keys()},
                           orient='index')

Districts_COUT = [i[0] for i in COUT.index]
Appeals_COUT = [i[1] for i in COUT.index]

COUT['DistrictCourts'] = Districts_COUT
COUT['AppealsCourts'] = Appeals_COUT

COUT = COUT[COUT[0] == 1.0]

#### Plot all Locations

In [ ]:
plt.scatter(Settlements['LON_X'],Settlements['LAT_Y'])
plt.scatter(Districts['LON_X'],Districts['LAT_Y'], color='Red', marker = 's')
plt.scatter(Appeals['LON_X'],Appeals['LAT_Y'], color='Green', marker = '^')
plt.show()

#### Plotting Assignments

In [ ]:
#Drawing Lines
plt.figure(figsize=(14,14))

#Plotting Points    
plt.scatter(Settlements['LON_X'],Settlements['LAT_Y'])
plt.scatter(Districts['LON_X'],Districts['LAT_Y'], color='Red', marker = 's',s=25)
plt.scatter(Appeals['LON_X'],Appeals['LAT_Y'], color='Green', marker = '^',s = 400)

for index,row in DOUT.iterrows():
    s = row['Settlement'] 
    d = row['DistrictCourts']
    #Get District number that Settlement is linked to
    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    Sett = Settlements.loc[Settlements['OBJECTID'] == s]
    X = [Sett.iloc[0,2],Dist.iloc[0,1]]
    Y = [Sett.iloc[0,1],Dist.iloc[0,2]]
    plt.scatter(Dist.iloc[0,1],Dist.iloc[0,2], color='Orange', marker = 's',s = 100)
    plt.plot(X,Y,zorder=1, color="Black")



#axes = plt.gca()
#axes.set_xlim([68.2,70])
#axes.set_ylim([34,35.5])

plt.show()

In [ ]:
#Draw Lines
plt.figure(figsize=(14,14))

for index,row in COUT.iterrows():
    d = row['DistrictCourts'] 
    a = row['AppealsCourts']

    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    App = Appeals.loc[Appeals['PROV_CODE'] == a]

    X = [Dist.iloc[0,1],App.iloc[0,1]]
    Y = [Dist.iloc[0,2],App.iloc[0,2]]

    plt.plot(X,Y,zorder=1, color="Black")
    
clr = cm.rainbow(np.linspace(0, 1, D))
for a in range(A):
    X = Appeals.iloc[a,1]
    Y = Appeals.iloc[a,2]
    plt.scatter(X,Y, color="Green", marker = '^')
    
for index,row in DOUT.iterrows():   
    d = row['DistrictCourts'] 
    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    X = Dist.iloc[0,1]
    Y = Dist.iloc[0,2]
    c = Dist.index[0]
    plt.scatter(X,Y, color=clr[c], marker = 's')
    
for index,row in DOUT.iterrows():
    s = row['Settlement'] 
    d = row['DistrictCourts']
    #Get District number that Settlement is linked to
    Dist = Districts.loc[Districts['DIST_CODE'] == d]
    Sett = Settlements.loc[Settlements['OBJECTID'] == s]

    c = Dist.index[0]
    X = Sett.iloc[0,2]                                                     
    Y = Sett.iloc[0,1]
    plt.scatter(X,Y, color=clr[c])
          
#axes = plt.gca()
#axes.set_xlim([68.2,70])
#axes.set_ylim([34,35.5])

sf = shp.Reader("Afghanistan_Districts","rb")
for shape in sf.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    plt.plot(x,y,color='k',linewidth=0.1)
plt.show()